# 生成所需数据

In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [2]:
# 指定工作空间
env.workspace = r'C:\Users\Runker\Desktop\ele_sb'
env.overwriteOutput = True

In [4]:
# 三调地块路径
sd_data = r'C:\Users\Runker\Desktop\ele_sb\gl_sd_dltb_sthd.shp'
# 母质路径
mz_data = r'C:\Users\Runker\Desktop\ele_sb\gl_dz_1km_polygon_20241010_river.shp'
# 坡位矢量路径
slope_data = r'C:\Users\Runker\Desktop\ele_sb\gl_slope_101_smooth_smooth.shp'

In [6]:
# 提取耕林园草
arcpy.Select_analysis(sd_data,"gl_sd_dlyc.shp",f"{'DLBM'} IN ('00','01', '02', '03','04','12')")

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\gl_sd_dlyc.shp'>

In [7]:
# 融合母质
arcpy.Dissolve_management(in_features=mz_data,out_feature_class='gl_dz_result_dissove.shp',dissolve_field='MZMC',multi_part='SINGLE_PART')

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\gl_dz_result_dissove.shp'>

In [8]:
# 相交
arcpy.Intersect_analysis(in_features=['gl_sd_dlyc','gl_dz_result_dissove',slope_data],out_feature_class='gl_merge_data',join_attributes='ALL')    

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\gl_merge_data.shp'>

In [9]:
# 转单部件
arcpy.MultipartToSinglepart_management('gl_merge_data','gl_merge_data_single')


<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\gl_merge_data_single.shp'>